Dataset collected from Kaggle: https://www.kaggle.com/ebiswas/imdb-review-dataset
IMDb Largest Review Dataset

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import dask.bag as db
# import json
import pandas as pd
client = Client() #Necessary in cases where task paralleling can be applied for Dask functions
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52013,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.91 GiB
Comm: tcp://192.168.1.105:52044,Total threads: 2
Dashboard: http://192.168.1.105:52045/status,Memory: 1.98 GiB
Nanny: tcp://127.0.0.1:52018,


In [2]:
%%writefile validation.yaml
#Update Validation.yaml file with details of the json file
file_type : json
file_name : imdb-part-01
columns :

Overwriting validation.yaml


In [3]:
#Load Configuration File & Confirm content
import yaml
with open('validation.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
config

{'file_type': 'json',
 'file_name': 'imdb-part-01',
 'columns': ['REVIEW ID',
  'REVIEWER',
  'MOVIE',
  'RATING',
  'REVIEW SUMMARY',
  'REVIEW DATE',
  'SPOILER TAG',
  'REVIEW DETAIL',
  'HELPFUL']}

In [4]:
# Re-write Column names on the dataset to standard format & compare against the declared clumns on the yaml file
def validate_columns(dataset, config):
    dataset = dataset.rename(columns = lambda str: str.replace('_',' ').upper()) 
    #check on missing columns in the dataset. If the dataset has more columns:
    if len(dataset.columns) > len(config['columns']):
        config_missing = [x for x in dataset.columns if x not in config['columns']]
        return print('The following columns are not found on the config file: \n{}'.format(config_missing))
    #if config file has more columns:
    elif len(dataset.columns) < len(config['columns']):
        dataset_missing = [x for x in config['columns'] if x not in dataset.columns]
        return print('The following columns are not found on the dataset: \n{}'.format(dataset_missing))
    # If columns are equal, confirm whether the column names are identical
    else:
        config['columns'].sort()
        if list(dataset.columns.sort_values()) != config['columns']:
            return print('The columns are not similar: \n Dataset: \t{} \nConfig File: \t{}'.format(list(dataset.columns.sort_values()), config['columns']))
        else: return('Column validation completed succesfully. All columns accounted for.')

In [5]:
#Load the dataset
imdb_part1 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])),
                            npartitions = 5)

In [6]:
#Validate columns in dataset against expected in yaml config file.
validate_columns(imdb_part1, config)

'Column validation completed succesfully. All columns accounted for.'

## Only to be utilized in test cases for verification

In [7]:
%%writefile validation.yaml
#Update yaml file for test scenario
file_type : json
file_name : imdb-part-02
columns :
          - REVIEW ID
          - REVIEWER
          - MOVIE
          - RATING
          - REVIEW SUMMARY
          - REVIEW DATE
          - SPOILER TAG
          - REVIEW DETAIL
          - HELPFUL

Overwriting validation.yaml


In [8]:
# Teast 1: Fail: Missing columns in config file
import random
imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
#Adding columns to dataset
imdb_part2 = imdb_part2.assign(New_Column = lambda x : random.randint(0,1000),
                                New_Column2 = lambda x : random.randint(2000,20000))
with open('validation.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
imdb_part2

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful,New_Column,New_Column2
npartitions=5,,,,,,,,,,,
0,object,object,object,float64,object,object,int64,object,object,int64,int64
202059,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
808236,...,...,...,...,...,...,...,...,...,...,...
1010292,...,...,...,...,...,...,...,...,...,...,...


In [9]:
validate_columns(imdb_part2, config)

The following columns are not found on the config file: 
['NEW COLUMN', 'NEW COLUMN2']


In [10]:
# Test 2: Fail: Missing columns in dataset.
imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
#Remove columns from dataset
imdb_part2 = imdb_part2.drop(['review_date', 'helpful'], axis = 1)
with open('validation.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
imdb_part2

,review_id,reviewer,movie,rating,review_summary,spoiler_tag,review_detail
npartitions=5,,,,,,,
0,object,object,object,float64,object,int64,object
202443,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...
809772,...,...,...,...,...,...,...
1012211,...,...,...,...,...,...,...


In [11]:
validate_columns(imdb_part2, config)

The following columns are not found on the dataset: 
['REVIEW DATE', 'HELPFUL']


In [13]:
# Test Case 3: Should be Successful
#Load Test file, update yaml config file in memory, & validate columns
imdb_part2 = dd.from_pandas(pd.read_json('datasets/iMDB_Reviews/{}.{}'.format(config['file_name'], config['file_type'])), npartitions = 5)
with open('validation.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
imdb_part2

,review_id,reviewer,movie,rating,review_summary,review_date,spoiler_tag,review_detail,helpful
npartitions=5,,,,,,,,,
0,object,object,object,float64,object,object,int64,object,object
202443,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
809772,...,...,...,...,...,...,...,...,...
1012211,...,...,...,...,...,...,...,...,...


In [14]:
validate_columns(imdb_part2, config)

'Column validation completed succesfully. All columns accounted for.'